# Tribal Lands

This layer shows areas designated as American Indian, Alaska Native and Native
Hawaiian Areas (**AIANNH**) by the U.S. Census in 2020. This dataset is updated on an annual basis as
part of the Census TIGER program and is available for download on the program's website. Areas
shown include: Alaska Native Regional Corporations, Tribal Subdivisions, Federal American Indian
Reservations, Off-Reservation Trust Lands, State American Indian Reservations, Hawaiian Home
Lands, Alaska Native Village Statistical Areas, Oklahoma Tribal Statistical Areas, State Designated
Tribal Statistical Areas, Tribal Designated Statistical Areas, and American Indian Joint-Use Areas.
Disclaimer: The data presented is “as is,” “as available” for informational purposes. NTIA does not
warrant the accuracy, adequacy, or completeness of this information and expressly disclaims any
liability for any errors or omissions.

In [1]:
import geopandas as gp
import pandas as pd

from elasticsearch import Elasticsearch, helpers
import json
from keplergl import KeplerGl
es = Elasticsearch(['https://3d6a9dd50c7c49c9ab5d23b6891bc03e.us-central1.gcp.cloud.es.io:9243'], 
                    http_auth=('elastic', 'WMzYk5RXyzE7MRShwPVwHzPX'), timeout=30)


/home/nhat/anaconda3/envs/py39/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.0-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
# ATTRIBUTES DEF @ page 22, 23
# https://nces.ed.gov/programs/edge/docs/TGRSHP2017_FILEDOC.pdf
# American Indian / Alaska Native / Native Hawaiian Areas (AIANNH) layer: https://www2.census.gov/geo/tiger/TIGER2020/AIANNH/
polygon = gp.read_file('tl_2020_us_aiannh.zip')
ttract = gp.read_file('https://www2.census.gov/geo/tiger/TIGER2020/TTRACT/tl_2020_us_ttract.zip')
polygon.shape, ttract.shape

((859, 16), (492, 11))

## Tribal polygon vs Tribal census tracts

In [6]:
kepler_cols =              [  
                # 'AIANNHCE'  ,        
                # 'AIANNHNS'  ,         
                'GEOID'  ,       
                'NAME'       ,    
                'NAMELSAD'   ,       
                # 'LSAD'        ,   
                # 'CLASSFP'     ,      
                # 'COMPTYP'    ,        
                # 'AIANNHR'     ,     
                'MTFCC'     ,      
                # 'FUNCSTAT'  ,        
                # 'ALAND'     ,    
                # 'AWATER'    ,    
                # 'INTPTLAT'   ,        
                # 'INTPTLON'  ,         
                'geometry'   , 
]
kepler_polygon = polygon[kepler_cols]
kelper_ttract = ttract[kepler_cols]
kepler_polygon.shape, kelper_ttract.shape

((859, 5), (492, 5))

In [ ]:
# load a saved Kepler config
config_name = 'config_tribalvs'
# LOAD the config  using jupyter %run magic ; Note the parameter escape {}
# i.e. execute variable assignment: config = ....
%run {config_name}.py
# variable my_config will contain saved kelper map settings

In [7]:
# Kepler map: 2 different tribal layers
map1 = KeplerGl(height=800, data={
    'polygon' : kepler_polygon, 
    'tract' : kelper_ttract,
}, config = my_config)

map1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'polygon':      GEOID                                               NAME  \
0    2320R         …

In [9]:
import pprint
# Save CURRENT Kepler config (all settings that have been done in Kelper Web Map UI) to a file
__config = map1.config
config_name = 'config_tribalvs'

# SAVE: pretty-write kepler map config to file
with open(f"{config_name}.py", 'w') as file:
    file.write(f"my_config = {pprint.pformat(__config)}")

## To ElasticSearch

In [ ]:
def create_tribal_index(es, ES_ROW_GEOID, index_name):
    create_request_body = {
       "settings": {
        "refresh_interval": "1s",
        "number_of_shards": 1,
        "number_of_replicas": 0
      },
        "mappings":{ 
            "properties":{
                'AIANNHCE'            : { "type":"text"},
                'AIANNHNS'            : { "type":"text"},
                ES_ROW_GEOID            : { "type":"text"},
                'NAME'            : { "type":"text"},
                'NAMELSAD'            : { "type":"text"},
                'LSAD'            : { "type":"text"},
                'CLASSFP'            : { "type":"text"},
                'COMPTYP'            : { "type":"text"},
                'AIANNHR'            : { "type":"text"},
                'MTFCC'            : { "type":"text"},
                'FUNCSTAT'            : { "type":"text"},
                'ALAND'           : { "type": "float"}, # an INT larger than ES integer type
                'AWATER'         : { "type": "float"}, # an INT larger than ES integer type
                'INTPTLAT'            : { "type":"text"},
                'INTPTLON'            : { "type":"text"},
                
                'location'    : {"type" : "geo_shape"},
            }
        }
    } 

    es.indices.create(index = index_name, body = create_request_body)
    print('created index ', index_name)

def upload_df(es, df, index_name, id_column, op_type):
    # """ This function upload plain json file, not geojson"""
    df = df.to_json(orient = 'records')
    json_records = json.loads(df)
    action_list = []
    for row in json_records:
        record = {
            '_op_type': op_type,
            '_index': index_name,
            # The type of the document
            "_type": '_doc', 
            '_id':row[id_column],
        }
        if op_type == 'update':
            record['doc_as_upsert'] = True
            record['doc'] = row
            # "doc": {"newkey": 'newvalue'},
        elif op_type == 'index':
            # a comma-separated list of the fields you want to retrieve.'
            record['_source'] = row
        else:
            print('upload_df only accepts `update` or `index`')
                   
        action_list.append(record)
        
    # bulk UPLOAD the records
    helpers.bulk(es, action_list)
    print(f"finished uploading {len(json_records)} records to {index_name}")
    

In [3]:
# Create Elasticsearch GEOMETRY column
geom = tribal['geometry']
geojson =  geom.to_json()
geojson_dict = json.loads(geojson)
tribal['location'] = [k['geometry'] for k in geojson_dict['features']]


In [5]:
index_name = "tribal_land"
ES_ROW_GEOID = 'GEOID'
tribal_ES_cols =              [  
                'AIANNHCE'  ,        
                'AIANNHNS'  ,         
                ES_ROW_GEOID  ,       
                'NAME'       ,    
                'NAMELSAD'   ,       
                'LSAD'        ,   
                'CLASSFP'     ,      
                'COMPTYP'    ,        
                'AIANNHR'     ,     
                'MTFCC'     ,      
                'FUNCSTAT'  ,        
                'ALAND'     ,    
                'AWATER'    ,    
                'INTPTLAT'   ,        
                'INTPTLON'  ,         
                'location'   , ]

# RUN ONCE
# create_tribal_index(es, ES_ROW_GEOID, index_name)

# RUN ONCE: NOTE: here using index _op_type: index = replace existing records
# WEIRD behavior: only successfully upload tribal[['GEOID', 'location']]
# REASON: cannot upload GeoDataFrame: type(tribal), type(tribal_ES),  # (geopandas.geodataframe.GeoDataFrame, pandas.core.frame.DataFrame)
tribal_ES = tribal[tribal_ES_cols]
tribal_ES.shape
# UPLOAD ONCE
# upload_df(es, tribal_ES, index_name, ES_ROW_GEOID, 'index')


(859, 16)